# Find the fastest way for getting information from depth files

## Loop

In [7]:
from lib import variable_alignment

class args:
    def __init__(self):

        self.input = '100_genomes.txt'
        self.gaps = 0.9
        self.mindepth = 5
        self.repeats = 'resources/regions_blindspots_modlin_farhat.bed'
        self.drug = 'resources/20160911_DR_filter_pos_reseqTB.txt'
        self.outg = ''
        self.suffix = "mutect2.filtered.homo.snps.vcf"
        self.threads = 1
        self.output_prefix = 'test'
        
args = args()

var_aln = variable_alignment.variable_alignment(args)


AttributeError: 'args' object has no attribute 'bed'

## Random access